2. Synchronisation de la copie de HAL
    * API en sortie xml-tei pour rechercher les notices nouvelles et les notices modifiées
        * C'est le critère "dateLastIndexed_tdate" qui est préféré à "dateModified_tdate" et à "dateReleased_tdate" pour couvrir les notices créées, les notices modifiées et celles qui sont réindexées pour d'autres raisons.
        * critère : q=dateLastIndexed_tdate:[NOW-1DAY%20TO%20NOW/HOUR]. Explication : [maintenant début de l'heure courante - 1 JOUR (=24h)] JUSQU'A [maintenant début de l'heure courante])
        * pour info : Pour tester avec dates exactes : api.archives-ouvertes.fr/search/INRIA2?q=dateLastIndexed_tdate:["2024-07-03T15:00:00.224Z"%20TO%20"2024-07-04T16:00:00.224Z"]&rows=1&wt=php&fl=docid,uri_s,dateLastIndexed_tdate&sort=docid%20asc&cursorMark=*
        * autres critères : rows=1 et cursorMark pour itérer sur chaque notice du résultat
        * notices
            * enregistrer chaque notice xml-tei (métadonnés de hal) correspondant à chaque docid (il y a un docid par version, donc cela permet d'enregistrer toutes les versions.
            * format du nom du fichier : halID_version.xml (exemple : hal-01044648_v1.xml)
            * dossier de stockage : /data/année/date_du_jour/Notices_Xml-tei_de_Hal
        * fichiers
            * Enregistrer uniquement le fichier PDF associé à la notice (l'objectif étant d'en extraire le full text dans un deuxième temps) dans un dossier correspondant à l'année et un sous-dossier correspondant au jour.
            * Critère d'identification des fichiers dans la notice : existence de balises \<ref\> dans \<edition\>, qui contiennent les liens vers tous les fichiers déposés dans la notice et aussi les liens externalLink
            * format du nom du fichier : " HALID_version_nom_du_fichier.extension du fichier" . exemple : inria-00441254_v2_NEL-splitting.pdf
            * exclusion :  tout ce qui est sous embargo.
            * Embargo : un fichier est créé qui liste les fichiers sous embargo avec la date.
            * Doublons : un fichier qui liste les pdf enregistrés permet d'éviter d'enregistrer deux fois le même fichier.
            * dossier de stockage : ../pdf_de_hal/année/date_du_jour
        

In [ ]:
###################################################
###################################################
## Ajout quotidien de notices nouvelles / modifiées,  et de leurs fichiers
#/!\ Retirer le filtre INRIA dans base_url pour la mise en prod!
#/!\ retirer la limite à 300 du compteur à la fin pour la mise en prod!
###################################################
###################################################

import requests
from lxml import etree
import time
import logging
import re
import os
import pandas as pd
from datetime import datetime, date
import gzip
import shutil
import urllib.parse
import openpyxl

# Installer lxml si besoin et importer
def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import lxml.etree as ET
except ImportError:
    print("Module 'lxml' non trouvé, installation en cours...")
    install_package("lxml")
    import lxml.etree as ET


# Dates actuelles
current_datetime = datetime.now()
date_extraction_current = current_datetime.strftime("%Y_%m_%d")
annee_current = current_datetime.strftime("%Y")
current_date = current_datetime.date()

# Répertoire de log
log_directory = '/data/log/'
os.makedirs(log_directory, exist_ok=True)
log_file = os.path.join(log_directory, f"{date_extraction_current}_log_synchro_part2_hal.txt")
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s', filemode='w')

# API HAL
base_url = "https://api.archives-ouvertes.fr/search/"
query = "dateLastIndexed_tdate:[NOW-1DAY TO NOW/HOUR]"
params = {"wt": "xml-tei", "rows": 1, "sort": "docid asc"}

# Variables
embargos = set()
compteur = affichage = compte_embargo = compte_externalLink = compteur_notices_enregistrees = compteur_telechargements = 0

# Création d'un index des notices xml existantes dans les sous-dossiers
index_notices_existantes = {}
for root, dirs, files in os.walk("/data/Notices_Xml-tei_de_Hal/"):
    for file in files:
        if file.endswith(".xml"):
            index_notices_existantes[file] = os.path.join(root, file)



# Liste des pdf téléchargés
pdf_registry_file = "/data/log/pdfs_deja_telecharges.txt.gz"

# Charger les URLs déjà téléchargées 
pdf_urls_enregistrees = set()
if os.path.exists(pdf_registry_file):
    with gzip.open(pdf_registry_file, 'rt', encoding='utf-8') as f:
        for line in f:
            url = line.strip()
            if url:
                pdf_urls_enregistrees.add(url)
else:
    print(f"{pdf_registry_file} non trouvé, création d'un nouveau fichier.")

# Dossiers
docid_folder_xml = f"/data/Notices_Xml-tei_de_Hal/{annee_current}/{date_extraction_current}"
docid_folder_pdf = f"/data/pdf_de_hal/{annee_current}/{date_extraction_current}/"
os.makedirs(docid_folder_xml, exist_ok=True)
os.makedirs(docid_folder_pdf, exist_ok=True)

cursor_mark = "*"
previous_cursor_mark = None
namespaces = {"tei": "http://www.tei-c.org/ns/1.0"}

# Boucle notice par notice
while cursor_mark != previous_cursor_mark:
    compteur += 1
    params["cursorMark"] = cursor_mark
    full_url = f"{base_url}?q={query}&wt={params['wt']}&rows={params['rows']}&sort={params['sort']}&cursorMark={params['cursorMark']}"
    response = requests.get(full_url)
    if response.status_code != 200:
        logging.error(f"Erreur: {response.status_code}")
        break

    try:
        data = etree.fromstring(response.content)
    except etree.XMLSyntaxError as e:
        logging.error(f"Erreur de parsing: {e}")
        break

    # Affichage du nombre de notices trouvées par l'API
    total_results = data.xpath('.//tei:measure[@commodity="totalSearchResults"]', namespaces=namespaces)
    if total_results and affichage == 0:
        #print(f"Nbre de notices indexées ces dernières 24h: {total_results[0].get('quantity')}")
        logging.info(f"Nbre de notices indexées ces dernières 24h: {total_results[0].get('quantity')}")
        affichage += 1
    
    # Récupération du "cursor_mark" pour la notice suivante
    next_cursor_mark = data.get("next")
    if not next_cursor_mark:
        break

    # Récpération du HalID, de la version (n_value)
    halID_element = data.xpath('.//tei:idno[@type="halId"]', namespaces=namespaces)
    if not halID_element:
        logging.error("HAL ID not found")
        break
    halID_value = halID_element[0].text
    #print(f"hal ID: {halID_value}")

    edition_current = data.xpath('.//tei:edition[@type="current"]', namespaces=namespaces)
    n_value = edition_current[0].get('n') if edition_current else ''

    #Définition du nom de fichier
    nom_fichier = os.path.join(docid_folder_xml, f'{halID_value}_{n_value}.xml')
    logging.info(f"notice: {halID_value}_{n_value}.xml")
    compteur_notices_enregistrees += 1

    nom_fichier_cible = f"{halID_value}_{n_value}.xml"
    if nom_fichier_cible in index_notices_existantes:
        try:
            os.remove(index_notices_existantes[nom_fichier_cible])
            logging.info(f"Ancienne notice supprimée : {index_notices_existantes[nom_fichier_cible]}")
        except Exception as e:
            logging.error(f"Erreur suppression fichier {index_notices_existantes[nom_fichier_cible]} : {e}")


# Enregistrement de la notice xml-tei
    with open(nom_fichier, "wb") as f:
        f.write(response.content)


    # Fonctions de nettoyage des noms de fichier
    # suppression d'un éventuel "?" et de ce qui suit dans le titre du fichier
    def clean_filename(filename):
        return filename.split('?')[0]

    # validation des caractères composant le nom de fichier
    def is_valid_filename(filename):
        return bool(re.match(r'.*\.[a-zA-Z]{3}$', filename))


    # Repérage des PDF et enregistrement
    refs = data.xpath('.//tei:edition[@type="current"]/tei:ref', namespaces=namespaces)

    # on spéciefie les types de fichiers qu'on va télécharger
    include_file_extension = ("pdf", "PDF")

    # Passage en revue de tous les liens vers un fichier
    for ref in refs:
        ref_type = ref.get('type')
        subtype = ref.get('subtype')
        n = ref.get('n')
        target = ref.get('target')
        date_tag = ref.find('tei:date', namespaces)

        # si le fichier ne termine pas par PDF et n'est pas de type "file", on passe à la suivante ref.
        if not target or not target.endswith(include_file_extension) or ref_type != 'file':
            continue

        # Identification d'un éventuel embargo
        embargo = 'False'
        embargomainfile = False

        if date_tag is not None:
            not_before_date = date_tag.get('notBefore')
            if not_before_date:
                ref_date = datetime.strptime(not_before_date, '%Y-%m-%d').date()
                # Si la date d'embargo est postérieure à la date du jour
                if ref_date > current_date:
                    embargo = 'True'
                    embargomainfile = ref_type == 'file'
                    embargos.add((ref_date, halID_value, embargomainfile, target))
                    logging.info(f"embargo à {ref_date} pour {halID_value}")
                    #print(f"embargo à {ref_date} pour {halID_value}")
                    continue

        
        file_name = clean_filename(os.path.basename(target))
        name, ext = os.path.splitext(file_name)

        # Limitation de la longueur du nom de fichier
        if len(file_name) > 100:
            file_name = f"{name[:97]}{ext}"

        pdf_filename = os.path.join(docid_folder_pdf, f"{halID_value}_{n_value}_main_{file_name}")

        if target in pdf_urls_enregistrees:
            continue  # déjà téléchargé dans une session précédente

        if is_valid_filename(pdf_filename):
            try:
                r = requests.get(target)
                r.raise_for_status()
                with open(pdf_filename, 'wb') as pdf_file:
                    pdf_file.write(r.content)
                    compteur_telechargements += 1
                    logging.info(f"fichier: {halID_value}_{n_value}_{file_name}")
                    #print(f'Téléchargé: {pdf_filename}')
                # ajout de l'url du fichier dans la liste des fichiers téléchargés
                pdf_urls_enregistrees.add(target)
                continue # si un pdf est enregistré, on passe à la notice suivante.
                    
            except Exception as e:
                logging.error(f"Erreur pour {halID_value}{n_value} - {ref_type} - {target}: {e}")

    # Définition du prochain cursor mark (pour la boucle)
    previous_cursor_mark = cursor_mark
    cursor_mark = next_cursor_mark

    # Limite pour test (18-20 juin 2025)
    if compteur > 1000:
        break

#Enregistrement des références de fichier sous embargo
#print(f"Liste des embargos : {embargos}")
compte_embargo = len(embargos)

embargo_file_path = "/data/log/Embargos.xlsx"
if os.path.exists(embargo_file_path):
    df_existing = pd.read_excel(embargo_file_path)
    df_new = pd.DataFrame(embargos, columns=['ref_date', 'halID_value','embargomainfile', 'target'])
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
else:
    df_combined = pd.DataFrame(embargos, columns=['ref_date', 'halID_value','embargomainfile', 'target'])

df_combined.to_excel(embargo_file_path, index=False)

with gzip.open(pdf_registry_file, 'wt', encoding='utf-8') as f:
    for url in pdf_urls_enregistrees:
        f.write(url + '\n')


logging.info(f"notices sous embargo : {compte_embargo}")
logging.info(f"nbre notices xml enregistrées : {compteur_notices_enregistrees}")
logging.info(f"nbre de fichiers enregistrés : {compteur_telechargements}")

print(f"notices sous embargo : {compte_embargo}")
print(f"nbre notices xml enregistrées : {compteur_notices_enregistrees}")
print(f"nbre de fichiers enregistrés : {compteur_telechargements}")

logging.info("Synchro 2e partie terminée.")
print("Synchro 2e partie terminée.")
